# Tech Intro

## Main tables

In [616]:
# Real Estate's objects
real_estate_list: list = [
    'ag_object',
    'ag_parse_ad',
    'ag_lead_object',
    'ag_presentation_object',
    'ag_object_platform',
    'ag_posting_ad_realty',
    'native_realty_filial',
    'foreign_realty_filial',
    'spr_metro',
    'spr_metro_connection',
    'ag_object_parking']
# Lead's objects
lead_list: list = [
    'leads',
    'lead_utm',
    'client_wishes',
    'leads_email_order',
    'request_from_site',
    'lead_finance',
    'leads_source_spr',
    'leads_calls_connection',
    'leads_pipeline',
    'leads_not_implemented_reason',
    'leads_status',
    'ag_pipeline',
    'ag_pipeline_status',
    'leads_meta',
    'leads_meta_fields',
    'leads_meta_group',
    'leads_contacts_connect',
    'leads_contact',
    'leads_contact_meta',
    'leads_contact_meta_fields']
# Deal's objects
deal_list: list = [
    # Deal
    'a_deal',
    'a_deal_meta',
    'deal_passport',
    'a_deal_finance',
    'a_deal_check',
    'a_deal_test',
    # Contractor
    'a_contractor',
    'a_contractor_meta',
    'a_deal_contractor',
    'a_deal_contractor_meta',
    # Object
    'a_object',
    'a_object_meta',
    'a_deal_object']

## Main Variables

In [617]:
# Address and name of your notebook
venv_path = "/home/bigdata/a.umarov/jupyter_P24"
# Exploratory Data Analysis(EDA) table (eda_table)
eda_table = deal_list[1] # You can generate full-EDA cycle if you want through the all of tables
                                # Just iterate it !
# Exploratory Data Analysis(EDA) obj as PySpark DataFrame
eda_df = None
# Set the date of checking
sample_data_date = '20221027'

## Useful command and modules (can be transform into the 'importlib'- smart import logic)

In [618]:
import os
import pandas as pd
# To data analyze need the module
import pyspark.pandas as ps
os.environ["PYSPARK_PYTHON"] = f"{venv_path}/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = f"{venv_path}/bin/python3"

import findspark
findspark.init("/opt/dev/spark/spark")
 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, DataFrameNaFunctions, \
                        Window, DataFrame, DataFrameStatFunctions, \
                        Column, Row, GroupedData, functions, types
from pyspark.sql.functions import isnan, when, count, col


conf = SparkConf().setAppName("get_info_from_" + eda_table + "_table")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
spark = SparkSession(sc)

W1120 00:59:10.518819 10390 process.cpp:2877] Attempted to spawn already running process version@10.62.4.11:44708
I1120 00:59:10.519548   547 sched.cpp:241] Version: 1.11.0
I1120 00:59:10.519852 10561 sched.cpp:345] New master detected at master@10.62.4.8:5050
I1120 00:59:10.519953 10561 sched.cpp:365] No credentials provided. Attempting to register without authentication
I1120 00:59:10.521515 10580 sched.cpp:753] Framework registered with 89161c75-8817-4ae4-ab24-cba26241d6f9-75606


## Read the spec-file from Hadoop and show some data sample

In [619]:
eda_df = spark.read.parquet("/prod/source/p24/" + eda_table + "/datekey=" + sample_data_date)
eda_df.show(10, False)

+-------+------+--------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|id     |deal  |key                                                     |value                                                                                                                                                                 |updated             |
+-------+------+--------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+
|1000005|100741|Эксклюзивность агентского договора                      |1                                                                                                                                            

## Show columns' properties

In [620]:
print('Data overview')
eda_df.printSchema()

Data overview
root
 |-- id: string (nullable = true)
 |-- deal: string (nullable = true)
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)
 |-- updated: string (nullable = true)



## Total rows (count)

In [621]:
eda_df.count()

3244006

## Simple func to execute and show the result of the SQL-instruction

In [622]:
def execute(select_instruction: str = "select count(*) from eda_sql") -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "select count(*) from eda_sql"
    :param :string select_instruction: SQL statement
    :return: result of the SQL-operation as a table
    """
    return spark.sql(select_instruction).show(truncate = False)

## Create the temp-table-view from the DF (called as "eda_sql")

In [623]:
eda_df.createOrReplaceTempView("eda_sql")

## Check the method and temp-table

In [624]:
execute("select count(*) from eda_sql")

+--------+
|count(1)|
+--------+
|3244006 |
+--------+



## Detect missing values and abnormal zeroes and print it

###### Check-rule Matrix


|  Name of the rule    |Available                   
|----------------------|-------------------------|
|NULL/NaN/NA           |+                        |
|'Zero'-value          |+                        |
|'Empty-string'        |+                        |

### 1st solution (PySpark & Pandas version)
#### Max perfomance but harder to read & understand

##### Filter and count after - all the values in the  our EDA-DF

In [625]:
# NULL/NaN/NA values condition
eda_df_null_values = eda_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_null_values

,id,deal,key,value,updated
0,0,0,0,9129,0


In [626]:
# Zero values condition
eda_df_zero_values = eda_df.select([count(when(col(c) == 0, c)).alias(c) \
                                    for c in eda_df.columns]).toPandas()
eda_df_zero_values

,id,deal,key,value,updated
0,0,0,0,4698,0


In [627]:
# Empty-string values condition
eda_df_empty_values = eda_df.select([count(when(col(c) == '', c)).alias(c) \
                                     for c in eda_df.columns]).toPandas()
eda_df_empty_values

,id,deal,key,value,updated
0,0,0,0,41339,0


In [628]:
# Unlock the limit to display all the columns
pd.options.display.max_rows = None
# To transpose a dataframe - use .T method
eda_df_pandas = pd.concat([eda_df_null_values, eda_df_zero_values, eda_df_empty_values], axis = 'index').T
# To rename columns to anything else after the transpose
eda_df_pandas.columns = (['NULL','Zero','Empty_string'])
eda_df_pandas

,NULL,Zero,Empty_string
id,0,0,0
deal,0,0,0
key,0,0,0
value,9129,4698,41339
updated,0,0,0


In [629]:
def set_result_to_percent(df: object = None) -> object:
    """
    Method to execute and show the result of your PandasDataFrame
    :default -> None-object
    :param :Pandas DF
    :return: DF after the conversion (absolute -> percent)
    """
    if df is None:
        return "Please incl. at least one Pandas DF as a parameter"
    return round(df * 100 / eda_df.count(), 3)

In [630]:
set_result_to_percent(eda_df_pandas)

,NULL,Zero,Empty_string
id,0.000,0.000,0.000
deal,0.000,0.000,0.000
key,0.000,0.000,0.000
value,0.281,0.145,1.274
updated,0.000,0.000,0.000


### SQL-instructions

In [631]:
#Find dupes or misunderstanding or useless static values 3 244 006 total
for col in eda_df.columns:
    execute("select count(distinct("+ col + ")) from eda_sql")

+------------------+
|count(DISTINCT id)|
+------------------+
|3244006           |
+------------------+



+--------------------+
|count(DISTINCT deal)|
+--------------------+
|190740              |
+--------------------+



+-------------------+
|count(DISTINCT key)|
+-------------------+
|51                 |
+-------------------+



+---------------------+
|count(DISTINCT value)|
+---------------------+
|85106                |
+---------------------+



+-----------------------+
|count(DISTINCT updated)|
+-----------------------+
|1738906                |
+-----------------------+



In [632]:
def execute_template(column: str ='id') -> object:
    """
    Method to execute and show the result of your SQL-instruction
    :default -> "ID incl."
    :param :string select_instruction: column name
    :return: result of the SQL-operation as a table
    """
    print(execute("SELECT DISTINCT(" + column + ")\
                FROM eda_sql\
                WHERE 1 = 1 \
                ORDER BY " + column + " DESC"))
    return execute("SELECT DISTINCT(" + column + ")\
                    FROM eda_sql\
                    WHERE 1 = 1 \
                    ORDER BY " + column + " ASC")

In [633]:
#Find dupes or misunderstanding or useless static values 3 244 006 total
for col in eda_df.columns:
    execute_template(col)

+------+
|id    |
+------+
|999999|
|999998|
|999997|
|999996|
|999995|
|999994|
|999993|
|999992|
|999991|
|999990|
|99999 |
|999989|
|999988|
|999987|
|999986|
|999985|
|999984|
|999983|
|999982|
|999981|
+------+
only showing top 20 rows

None


+-------+
|id     |
+-------+
|1000   |
|10000  |
|100000 |
|1000000|
|1000001|
|1000002|
|1000003|
|1000004|
|1000005|
|1000006|
|1000007|
|1000008|
|1000009|
|100001 |
|1000010|
|1000011|
|1000012|
|1000013|
|1000014|
|1000015|
+-------+
only showing top 20 rows



+-----+
|deal |
+-----+
|99999|
|99998|
|99997|
|99996|
|99995|
|99994|
|99993|
|99992|
|99991|
|99990|
|9999 |
|99989|
|99988|
|99987|
|99986|
|99985|
|99984|
|99983|
|99982|
|99981|
+-----+
only showing top 20 rows

None


+------+
|deal  |
+------+
|100   |
|1000  |
|10000 |
|100000|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|10001 |
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
+------+
only showing top 20 rows



+---------------------------------------------------+
|key                                                |
+---------------------------------------------------+
|представитель фио                                  |
|в лице (доверенный сотрудник)                      |
|Юрист                                              |
|Эксклюзивность агентского договора                 |
|Цена договора является окончательной и изменению   |
|ФИО Найденного лица                                |
|Условия о неустойке в агентском договоре           |
|Условие о гарантиях                                |
|Сумма кредита                                      |
|Статус                                             |
|Срок принятия\передачи недвижимости                |
|Срок кредита                                       |
|Срок заключения ДКП                                |
|Срок действия агентского договора                  |
|Совместная сделка                                  |
|Сведения о найденном лице  

+--------------------------------------------------------------------------------------------+
|key                                                                                         |
+--------------------------------------------------------------------------------------------+
|% ставка по кредиту                                                                         |
|Cтраховая компания                                                                          |
|Банк (по ипотеке)                                                                           |
|Дата акта приема-передачи во исполнение договора об отчуждении объекта недвижимого имущества|
|Дата заключения ДКП                                                                         |
|Дата заключения ПДКП                                                                        |
|Дата заключения агентского договора                                                         |
|Дата открытия карточки                           

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                            |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|№99266521                                                                                                                                                        |
|№37725702 совместная сделка. документы там \r\n                                                                                                                  |
|№ 73:24:041904:282-73/049/2018-1  от 31.07.2018  (Залог в силу закона)                                                                                           |
|№ 73:24:041410:

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

+--------------------+
|updated             |
+--------------------+
|2022-10-27T00:26:07Z|
|2022-10-27T00:25:42Z|
|2022-10-27T00:25:41Z|
|2022-10-26T18:25:03Z|
|2022-10-26T18:13:24Z|
|2022-10-26T17:22:51Z|
|2022-10-26T17:03:40Z|
|2022-10-26T17:03:19Z|
|2022-10-26T16:41:32Z|
|2022-10-26T16:17:05Z|
|2022-10-26T16:16:01Z|
|2022-10-26T16:14:06Z|
|2022-10-26T16:14:02Z|
|2022-10-26T16:11:14Z|
|2022-10-26T15:56:25Z|
|2022-10-26T15:55:55Z|
|2022-10-26T15:55:04Z|
|2022-10-26T15:55:03Z|
|2022-10-26T15:54:25Z|
|2022-10-26T15:54:24Z|
+--------------------+
only showing top 20 rows

None


+--------------------+
|updated             |
+--------------------+
|2017-02-06T09:11:16Z|
|2017-02-06T09:12:34Z|
|2017-02-06T09:12:41Z|
|2017-02-06T09:15:07Z|
|2017-02-06T09:15:28Z|
|2017-02-06T09:15:43Z|
|2017-02-06T09:15:58Z|
|2017-02-06T09:16:07Z|
|2017-02-06T09:16:23Z|
|2017-02-06T09:16:31Z|
|2017-02-06T09:16:52Z|
|2017-02-06T09:17:26Z|
|2017-02-06T09:17:42Z|
|2017-02-06T09:23:43Z|
|2017-02-06T13:22:23Z|
|2017-02-07T03:23:58Z|
|2017-02-07T03:24:53Z|
|2017-02-09T10:00:44Z|
|2017-02-09T10:01:46Z|
|2017-02-09T10:01:58Z|
+--------------------+
only showing top 20 rows



## Bonus

### 2nd solution (Cycle and variables)
#### Slower but more simple and has more readability than the previous method 

### Create the list of variables to re-use if needed

In [ ]:
useless_columns = []
null_values = []
zero_values = []
empty_string = []
col_name = []

In [ ]:
for col in eda_df.columns:
    null = round((eda_df.filter(eda_df[col].isNull()).count()/eda_df.count())*100, 3)
    zero = round(eda_df.filter(eda_df[col]==0).count()/eda_df.count()*100, 3)
    empty = round(eda_df.filter(eda_df[col]=='').count()/eda_df.count()*100, 3)
    if null in [0, 100] and zero in [0, 100] and empty in [0, 100]:
        useless_columns.append(col)
        continue
    null_values.append(null)
    zero_values.append(zero)
    empty_string.append(empty)
    col_name.append(col)

In [ ]:
zipped = list(zip(null_values, zero_values, empty_string))
eda_df_pandas2 = pd.DataFrame(zipped, index=col_name, columns=['NULL', 'Zero', 'Empty'])
eda_df_pandas2

### Remove useless columns

In [ ]:
eda_df = eda_df.drop(*useless_columns)

In [ ]:
pd.DataFrame(eda_df.dtypes, columns = ['Column Name','Data type'])

In [615]:
spark.stop()

I1120 00:58:34.399523 10390 sched.cpp:2174] Asked to stop the driver
I1120 00:58:34.399699 10531 sched.cpp:1207] Stopping framework 89161c75-8817-4ae4-ab24-cba26241d6f9-75605
